In [1]:
import heapq
from collections import deque
from random import choice
from collections import namedtuple


# Define constants
PUZZLE_DIM = 3
GOAL_STATE = tuple(range(1, PUZZLE_DIM * PUZZLE_DIM)) + (0,)
Action = namedtuple('Action', ['pos1', 'pos2'])


In [2]:
# Helper functions
def get_neighbors(state):
    """
    Get possible moves from current state.
    """
    neighbors = []
    zero_index = state.index(0)
    row, col = zero_index // PUZZLE_DIM, zero_index % PUZZLE_DIM
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    for dr, dc in directions:
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < PUZZLE_DIM and 0 <= new_col < PUZZLE_DIM:
            new_index = new_row * PUZZLE_DIM + new_col
            new_state = list(state)
            new_state[zero_index], new_state[new_index] = new_state[new_index], new_state[zero_index]
            neighbors.append((tuple(new_state), (dr, dc)))
    
    return neighbors

In [3]:
# Function to shuffle the puzzle to generate a randomized initial state
def shuffle_puzzle(goal_state, randomize_steps):
    start_state = list(goal_state)
    for _ in range(randomize_steps):
        neighbors = get_neighbors(tuple(start_state))
        start_state, _ = choice(neighbors)
    return tuple(start_state)


# Function to print the puzzle as a matrix
def print_puzzle(state):
    """
    Prints the puzzle state as a 3x3 grid.
    """
    for i in range(PUZZLE_DIM):
        print(state[i * PUZZLE_DIM:(i + 1) * PUZZLE_DIM])
    print()

In [4]:
# Breadth-First Search (BFS) Solution
def bfs_solve_puzzle(start, goal):
    """
    Solves the n^2-1 puzzle using BFS and returns the sequence of moves.
    """
    queue = deque([(start, [])])
    visited = set()
    visited.add(start)
    
    while queue:
        current_state, path = queue.popleft()
        if current_state == goal:
            return path
        for neighbor_state, move in get_neighbors(current_state):
            if neighbor_state not in visited:
                visited.add(neighbor_state)
                queue.append((neighbor_state, path + [move]))
    return None

In [5]:
# A* Search Solution with Enhanced Heuristic
def manhattan_distance(state):
    """
    Calculate the Manhattan Distance heuristic for the given state.
    """
    distance = 0
    for index, value in enumerate(state):
        if value == 0:
            continue
        target_row, target_col = (value - 1) // PUZZLE_DIM, (value - 1) % PUZZLE_DIM
        current_row, current_col = index // PUZZLE_DIM, index % PUZZLE_DIM
        distance += abs(target_row - current_row) + abs(target_col - current_col)
    return distance

In [6]:

def a_star_solve_puzzle(start, goal):
    """
    Solves the n^2-1 puzzle using A* with Manhattan heuristic.
    """
    priority_queue = []
    heapq.heappush(priority_queue, (manhattan_distance(start), 0, start, []))
    visited = set()
    visited.add(start)
    
    while priority_queue:
        _, cost, current_state, path = heapq.heappop(priority_queue)
        if current_state == goal:
            return path
        for neighbor_state, move in get_neighbors(current_state):
            if neighbor_state not in visited:
                visited.add(neighbor_state)
                new_cost = cost + 1
                heapq.heappush(priority_queue, (new_cost + manhattan_distance(neighbor_state), new_cost, neighbor_state, path + [move]))
    return None

In [7]:
# Depth-First Search (DFS) Solution
def dfs_solve_puzzle(start, goal, depth_limit=None):
    """
    Solves the n^2-1 puzzle using DFS and returns the sequence of moves.
    """
    stack = [(start, [], 0)]
    visited = set()
    
    while stack:
        current_state, path, depth = stack.pop()
        if current_state == goal:
            return path
        visited.add(current_state)
        if depth_limit is not None and depth >= depth_limit:
            continue
        for neighbor_state, move in reversed(get_neighbors(current_state)):
            if neighbor_state not in visited:
                stack.append((neighbor_state, path + [move], depth + 1))
    return None

In [8]:

# Example Usage
if __name__ == "__main__":
    # Generate a randomized start state
    start_state = shuffle_puzzle(GOAL_STATE, randomize_steps=1000)
    print("Initial state:")
    print_puzzle(start_state)
    print("Goal state:")
    print_puzzle(GOAL_STATE)

    # BFS Solution
    bfs_path = bfs_solve_puzzle(start_state, GOAL_STATE)
    print("\nBFS Solution Path:", bfs_path)
    print("BFS Solution Length:", len(bfs_path) if bfs_path else "No solution found")

    # A* Solution
    a_star_path = a_star_solve_puzzle(start_state, GOAL_STATE)
    print("\nA* Solution Path:", a_star_path)
    print("A* Solution Length:", len(a_star_path) if a_star_path else "No solution found")
    print("A* Estimated Cost (Manhattan Heuristic):", manhattan_distance(start_state))

    # DFS Solution
    dfs_path = dfs_solve_puzzle(start_state, GOAL_STATE, depth_limit=20)
    print("\nDFS Solution Path:", dfs_path)
    print("DFS Solution Length:", len(dfs_path) if dfs_path else "No solution found")

    # Additional Output for Analysis
    print("\n--- Analysis ---")
    print("BFS explored nodes to find solution:", len(bfs_path) if bfs_path else "No solution found")
    print("A* explored nodes to find solution (estimated):", len(a_star_path) if a_star_path else "No solution found")
    print("DFS explored nodes with depth limit 20:", len(dfs_path) if dfs_path else "No solution found")

Initial state:
(0, 1, 4)
(5, 8, 3)
(2, 7, 6)

Goal state:
(1, 2, 3)
(4, 5, 6)
(7, 8, 0)


BFS Solution Path: [(1, 0), (1, 0), (0, 1), (-1, 0), (-1, 0), (0, -1), (1, 0), (0, 1), (-1, 0), (0, 1), (1, 0), (0, -1), (-1, 0), (0, -1), (1, 0), (0, 1), (0, 1), (1, 0)]
BFS Solution Length: 18

A* Solution Path: [(1, 0), (1, 0), (0, 1), (-1, 0), (-1, 0), (0, -1), (1, 0), (0, 1), (-1, 0), (0, 1), (1, 0), (0, -1), (-1, 0), (0, -1), (1, 0), (0, 1), (0, 1), (1, 0)]
A* Solution Length: 18
A* Estimated Cost (Manhattan Heuristic): 12

DFS Solution Path: None
DFS Solution Length: No solution found

--- Analysis ---
BFS explored nodes to find solution: 18
A* explored nodes to find solution (estimated): 18
DFS explored nodes with depth limit 20: No solution found
